# 本チャプターの目次
1. ビジネスメタデータの重要な役割
2. ビジネスメタデータの活用方法
3. 代表的なビジネスメタデータは？
4. 実際にビジネスメタデータを覗いてみよう
5. メタデータ保存用のテーブルを作成してみよう

# ビジネスメタデータの重要な役割

データマネジメントとは、

# ビジネスメタデータの活用方法


# 代表的なビジネスメタデータは？

# 実際にビジネスメタデータを覗いてみよう

# メタデータ保存用のテーブルを作成してみよう

最終的にメタデータをMysqlに保存する目的で本コースでは以下の順番で対応していきます。

1. テクニカルメタデータをSparkのテーブルに一度保存
2. オペレーショナルメタデータをSparkのテーブルに一度保存
3. 1,2とビジネスメタデータと合わせてMysqlのテーブルに保存

そのための準備を行います。

メタデータ保存用のメタデータの保存用のテーブルとMysqlのテーブルをここで作成します。

Sparkテーブル(metadata_tmpデータベース)
- database_name -> データベース名
- table_name -> テーブル名
- table_definition ->テーブル定義
- sammary ->　テーブル説明
- row_num ->　レコード件数
- selectivity ->　セレクティビティ
- consistency_flag ->　コンシステンシー
- frequency_access ->　アクセス数

Mysqlテーブル(metadataデータベース)
- database_name -> Sparkデータベース名
- table_name -> Sparkテーブル名
- table_definition -> Sparkテーブル定義
- sammary ->　Sparkテーブル説明
- row_num ->　レコード件数
- selectivity ->　セレクティビティ
- consistency_flag ->　コンシステンシー
- frequency_access ->　アクセス数

# メタデータ保存用のMysqlテーブル

```
CREATE DATABASE if not exists metadata;
USE metadata;
CREATE TABLE if not exists metadatas(
    database_name VARCHAR(255) , 
    table_name VARCHAR(255) , 
    table_definition VARCHAR(255) , 
    sammary VARCHAR(255) , 
    row_num VARCHAR(255) , 
    selectivity VARCHAR(255) , 
    consistency_flag VARCHAR(255) , 
    frequency_access VARCHAR(255) ,
    PRIMARY KEY (database_name,table_name));
```

コーンソールに移動してMysqlコマンドを実行します。

In [ ]:
# メタデータ保存用のSparkテーブル
spark.sql("create database if not exists metadata_tmp")
spark.sql(""" 
CREATE TABLE IF NOT EXISTS metadata_tmp.sample_metadata (database_name String, table_name String,table_definition String,sammary String,row_num String,selectivity String,consistency_flag String,frequency_access String)
PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/parquet/';
""")

In [ ]:
#　テーブルができたか確認します
spark.sql("show tables in metadata_tmp").show() 

In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()


In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

In [ ]:
spark.stop()
spark.sparkContext.stop()